In [33]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../numeral_context/')
sys.path.append('../../../')
sys.path.append('../../../utils/')
sys.path.append('../../mag_num/')
sys.path.append('../numeral_prediction/')
import pickle
import numpy as np
import pandas as pd
from number_handler import is_numeral, to_numeral
from tqdm import tqdm_notebook
from create_abstract_embeddings import get_all_target_numerals, random_sample_numerals, create_magnitude_embeds
from numeral_prediction_evaluator import Evaluator
from numeral_to_embed import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
dataset = pickle.load(open('../numeral_prediction/data.pkl','rb'))
res = get_all_target_numerals(dataset)
sampled_res = random_sample_numerals(res)
pickle.dump(sampled_res, open('./magnitudes_numerals.pkl', 'wb'))
magnitudes_numerals = pickle.load(open('./magnitudes_numerals.pkl', 'rb'))

In [66]:
word2idx = pickle.load(open('../exps/title_numeracy_embed/word2idx.pkl', 'rb'))
e1 = Evaluator(
    word2idx,
    dataset[:30000],
)

In [67]:
result_dict = {}

In [71]:
for i in ['50', '100', '150', '200', '300']:
    
    trained_prototypes = pickle.load(open('../../../data/numeracy600k/save/som-{}-0/trained_prototypes_epoch5_{}_1.0.dat'.format(i, i),'rb'))
    embeds = [load_prototype(i, trained_prototypes, log_space=False) for i in magnitudes_numerals]
    numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim
#     pickle.dump(numeral_embed_i, open('./p{}00_i.emb'.format(i), 'wb'))
#     pickle.dump(numeral_embed_o, open('./p{}00_o.emb'.format(i), 'wb'))
    idx2vec_i = pickle.load(open('../exps/title_numeracy_embed/embed/p-{}_i.emb'.format(i), 'rb'))
    idx2vec_o = pickle.load(open('../exps/title_numeracy_embed/embed/p-{}_o.emb'.format(i), 'rb'))
    e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
    e1.load_idx2vec(idx2vec_i, idx2vec_o)
    e1.compute_norm_factor_all(
        pickle.load(open('../../../data/numeracy600k/save/som-{}-0/idx2vec_o_epoch5.dat'.format(i), 'rb'))
    )
    ranks = e1.evaluate_avg_rank()
    micro_f1, macro_f1 = e1.evaluate_f1()
    result_dict['p-{}'.format(i)] = [ranks, micro_f1, macro_f1]

In [77]:
for i in ['50', '100', '150', '200', '300']:
    trained_prototypes = pickle.load(open('../../../data/numeracy600k/save-log/som-{}-0/trained_prototypes_epoch5_{}_1.0.dat'.format(i, i),'rb'))
    embeds = [load_prototype(i, trained_prototypes, log_space=True) for i in magnitudes_numerals]
    numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim
#     pickle.dump(numeral_embed_i, open('./p{}00_i.emb'.format(i), 'wb'))
#     pickle.dump(numeral_embed_o, open('./p{}00_o.emb'.format(i), 'wb'))
    idx2vec_i = pickle.load(open('../exps/title_numeracy_embed/embed/p-{}-log_i.emb'.format(i), 'rb'))
    idx2vec_o = pickle.load(open('../exps/title_numeracy_embed/embed/p-{}-log_o.emb'.format(i), 'rb'))

    e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
    e1.load_idx2vec(idx2vec_i, idx2vec_o)
    e1.compute_norm_factor_all(
        pickle.load(open('../../../data/numeracy600k/save-log/som-{}-0/idx2vec_o_epoch5.dat'.format(i), 'rb'))
    )
    ranks = e1.evaluate_avg_rank()
    micro_f1, macro_f1 = e1.evaluate_f1()
    result_dict['p-{}-log'.format(i)] = [ranks, micro_f1, macro_f1]

In [78]:
for i in ['50', '100', '150', '200', '300']:
    for j in ['hard', 'soft']:
        trained_prototypes = pickle.load(open('../../../data/numeracy600k/save/gmm-{}-rd-{}-0/trained_gmms_epoch5_{}.dat'.format(i, j, i),'rb'))
        gmm = pickle.load(open('../../../data/numeracy600k/NumeralAsNumeral/gmm/gmm-{}-rd-{}.dat'.format(i, j),'rb'))

        embeds = [load_GMM(i, trained_prototypes, gmm) for i in magnitudes_numerals]
        numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

        idx2vec_i = pickle.load(open('../exps/title_numeracy_embed/embed/g-{}-{}_i.emb'.format(i, j), 'rb'))
        idx2vec_o = pickle.load(open('../exps/title_numeracy_embed/embed/g-{}-{}_o.emb'.format(i, j), 'rb'))

        e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
        e1.load_idx2vec(idx2vec_i, idx2vec_o)
        e1.compute_norm_factor_all(
            pickle.load(open('../../../data/numeracy600k/save/gmm-{}-rd-{}-0/idx2vec_o_epoch5.dat'.format(i, j), 'rb'))
        )
        ranks = e1.evaluate_avg_rank()
        micro_f1, macro_f1 = e1.evaluate_f1()
        result_dict['g-{}-{}'.format(i, j)] = [ranks, micro_f1, macro_f1]

In [79]:
for i in ['50', '100', '150', '200', '300']:
    for j in ['hard', 'soft']:
        trained_prototypes = pickle.load(open('../../../data/numeracy600k/save-log/gmm-{}-rd-{}-0/trained_gmms_epoch5_{}.dat'.format(i, j, i),'rb'))
        gmm = pickle.load(open('../../../data/numeracy600k/NumeralAsNumeral/gmm_log/gmm-{}-rd-{}.dat'.format(i, j),'rb'))

        embeds = [load_GMM(i, trained_prototypes, gmm, log_space=True) for i in magnitudes_numerals]
        numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

        idx2vec_i = pickle.load(open('../exps/title_numeracy_embed/embed/g-{}-{}-log_i.emb'.format(i, j), 'rb'))
        idx2vec_o = pickle.load(open('../exps/title_numeracy_embed/embed/g-{}-{}-log_o.emb'.format(i, j), 'rb'))

        e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
        e1.load_idx2vec(idx2vec_i, idx2vec_o)
        e1.compute_norm_factor_all(
            pickle.load(open('../../../data/numeracy600k/save-log/gmm-{}-rd-{}-0/idx2vec_o_epoch5.dat'.format(i, j), 'rb'))
        )
        ranks = e1.evaluate_avg_rank()
        micro_f1, macro_f1 = e1.evaluate_f1()
        result_dict['g-{}-{}-log'.format(i, j)] = [ranks, micro_f1, macro_f1]

In [84]:
ivec = pickle.load(open('../../../data/numeracy600k/save/NumeralAsTokenUnkNumeral-0-550/idx2vec_i_epoch5.dat','rb'))
ovec = pickle.load(open('../../../data/numeracy600k/save/NumeralAsTokenUnkNumeral-0-550/idx2vec_o_epoch5.dat','rb'))
w2i = pickle.load(open('../../../data/numeracy600k/NumeralAsTokenUnkNumeral/word2idx.dat','rb'))
embeds = [load_TOKEN(i, ivec, ovec, w2i) for i in magnitudes_numerals]
numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

idx2vec_i = pickle.load(open('../exps/title_numeracy_embed/embed/token_i.emb', 'rb'))
idx2vec_o = pickle.load(open('../exps/title_numeracy_embed/embed/token_o.emb', 'rb'))

e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
e1.load_idx2vec(idx2vec_i, idx2vec_o)
e1.compute_norm_factor_all(
    pickle.load(open('../../../data/numeracy600k/save/NumeralAsTokenUnkNumeral-0-550/idx2vec_o_epoch5.dat', 'rb'))
)
ranks = e1.evaluate_avg_rank()
micro_f1, macro_f1 = e1.evaluate_f1()
result_dict['token'] =  [ranks, micro_f1, macro_f1]

In [86]:
model_path = '../../../data/numeracy600k/save/LSTM-0/sgns_epoch1.pt'
# embeds = [load_LSTM(i, model_path, 300) for i in magnitudes_numerals]
# numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim
# pickle.dump(numeral_embed_i, open('./LSTM_i.emb', 'wb'))
# pickle.dump(numeral_embed_o, open('./LSTM_o.emb', 'wb'))
numeral_embed_i = pickle.load(open('./LSTM_i.emb', 'rb'))
numeral_embed_o = pickle.load(open('./LSTM_o.emb', 'rb'))
idx2vec_i = pickle.load(open('../exps/title_numeracy_embed/embed/LSTM_i.emb', 'rb'))
idx2vec_o = pickle.load(open('../exps/title_numeracy_embed/embed/LSTM_o.emb', 'rb'))

e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
e1.load_idx2vec(idx2vec_i, idx2vec_o)
e1.compute_norm_factor_all(
    pickle.load(open('../../../data/numeracy600k/save/LSTM-0/idx2vec_o_epoch5.dat', 'rb'))
)
ranks = e1.evaluate_avg_rank()
micro_f1, macro_f1 = e1.evaluate_f1()
result_dict['LSTM'] =  [ranks, micro_f1, macro_f1]

In [95]:
model_path = '../../../data/numeracy600k/save/LSTM-1/sgns_epoch1.pt'
embeds = [load_LSTM(i, model_path, 300) for i in magnitudes_numerals]
numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim
pickle.dump(numeral_embed_i, open('./LSTM1_i.emb', 'wb'))
pickle.dump(numeral_embed_o, open('./LSTM1_o.emb', 'wb'))
# numeral_embed_i = pickle.load(open('./LSTM1_i.emb', 'rb'))
# numeral_embed_o = pickle.load(open('./LSTM1_o.emb', 'rb'))
idx2vec_i = pickle.load(open('../exps/title_numeracy_embed/embed/LSTM1_i.emb', 'rb'))
idx2vec_o = pickle.load(open('../exps/title_numeracy_embed/embed/LSTM1_o.emb', 'rb'))

e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
e1.load_idx2vec(idx2vec_i, idx2vec_o)
e1.compute_norm_factor_all(
    pickle.load(open('../../../data/numeracy600k/save/LSTM-1/idx2vec_o_epoch5.dat', 'rb'))
)
ranks = e1.evaluate_avg_rank()
micro_f1, macro_f1 = e1.evaluate_f1()
result_dict['LSTM1'] =  [ranks, micro_f1, macro_f1]

FileNotFoundError: [Errno 2] No such file or directory: '../exps/title_numeracy_embed/embed/LSTM1_o.emb'

In [87]:
embeds = [load_fixed(i, 300) for i in magnitudes_numerals]
numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

idx2vec_i = pickle.load(open('../exps/title_numeracy_embed/embed/Fixed_i.emb', 'rb'))
idx2vec_o = pickle.load(open('../exps/title_numeracy_embed/embed/Fixed_o.emb', 'rb'))

e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
e1.load_idx2vec(idx2vec_i, idx2vec_o)
e1.compute_norm_factor_all(
    pickle.load(open('../../../data/numeracy600k/save/fixed-0/idx2vec_o_epoch5.dat'.format(i, j), 'rb'))
)
ranks = e1.evaluate_avg_rank()
micro_f1, macro_f1 = e1.evaluate_f1()
result_dict['Fixed'] =  [ranks, micro_f1, macro_f1]

In [32]:
result_dict

{'p-200': [array([2.65685, 2.2503 , 2.21645]),
  array([0.3458 , 0.31095, 0.33655]),
  array([0.24429344, 0.17396251, 0.22830152])],
 'p-300': [array([2.6939 , 2.13285, 2.10785]),
  array([0.34695, 0.3427 , 0.3672 ]),
  array([0.2074501 , 0.30261152, 0.26727977])],
 'p-500': [array([2.646  , 2.1577 , 2.12285]),
  array([0.35155, 0.33305, 0.3612 ]),
  array([0.25039404, 0.28640396, 0.261676  ])],
 'p-200-log': [array([2.6702, 2.135 , 2.1071]),
  array([0.3567 , 0.35155, 0.3717 ]),
  array([0.21154131, 0.32423792, 0.27459233])],
 'p-300-log': [array([2.5584 , 2.1709 , 2.13635]),
  array([0.3565 , 0.32545, 0.35   ]),
  array([0.24730029, 0.26305895, 0.2420365 ])],
 'p-500-log': [array([2.6354 , 2.1582 , 2.12305]),
  array([0.35555, 0.3379 , 0.36195]),
  array([0.20881762, 0.29728428, 0.26095905])],
 'g-300-hard': [array([2.65855, 2.20785, 2.183  ]),
  array([0.35835, 0.3345 , 0.35795]),
  array([0.22017677, 0.27963035, 0.25489275])],
 'g-300-soft': [array([2.6   , 2.1822, 2.1519]),
  arra

In [94]:
e1.show_scores(result_dict, 3).T

,0,1,2
p-50,2.105933,0.474767,0.314232
p-100,1.925400,0.473067,0.298584
p-150,1.887433,0.478767,0.298050
p-200,1.885733,0.476933,0.290096
p-300,1.885000,0.482833,0.313233
p-50-log,1.890367,0.481700,0.293423
p-100-log,1.910100,0.478467,0.296583
p-150-log,1.927667,0.471867,0.286041
p-200-log,1.918033,0.477333,0.337644
p-300-log,1.903933,0.480667,0.299941
